In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 25.2MB/s 
     |████████████████████████████████| 901kB 41.3MB/s 
     |████████████████████████████████| 3.3MB 39.7MB/s 


In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import transformers as tf
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
data_sms= pd.read_table('sample_data/SMSSpamCollection',
                   sep='\t',
                   header=None,
                   names=['label', 'sms'])

data_sms.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
sample_data = data_sms[:1000]

In [36]:
ml_cls, tokenizer_cls, pt_weights = (tf.DistilBertModel, tf.DistilBertTokenizer, 'distilbert-base-uncased')

In [37]:
tzr = tokenizer_cls.from_pretrained(pt_weights)

In [39]:
mod = ml_cls.from_pretrained(pt_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
tokenized = sample_data['sms'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [41]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [42]:
np.array(padded).shape

(1000, 100)

In [43]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 100)

Torch.no_grad() deactivates autograd engine. Eventually it will reduce the memory usage and speed up computations.

Use of Torch.no_grad():

    To perform inference without Gradient Calculation.

    To make sure there's no leak test data into the model.

It's generally used to perform Validation. Reason in this case one can use validation batch of large size

In [44]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [45]:
features = last_hidden_states[0][:,0,:].numpy()

In [46]:
labels = sample_data['label']

In [47]:
#mapping = {'ham' : 0 , 'spam':1}
labels = labels.map(mapping)

In [48]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [49]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_features, train_labels)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [50]:
clf.score(test_features, test_labels)

0.92